In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gmaps
import gmaps.datasets
import ipyleaflet
from IPython.display import display

In [ ]:
#importing environment variables
from dotenv import dotenv_values
env_variables = dotenv_values('Ignore.env')

In [ ]:
df = pd.read_csv('/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Intermediate Data/Wrangled_data')

In [ ]:
df.head()

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)


In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
#Filling in all NAs from the different sales categories with 0's
df[['SaleNom', 'Baked Goods', 'Beverages', 'Classes and Events',
       'Condiments + Sauces', 'Dairy', 'Dried Herbs + Spices', 'Eggs',
       'Grains, Flours, Cereal + Pastas', 'Handmade Home Goods + Gifts',
       'Honey, Syrups, Jams + Jellies', 'Iowa Food Co-op Shop',
       'Local Produce', 'Meat - Beef', 'Meat - Chicken + Capon', 'Meat - Pork',
       'Meats - Other', 'Non-Food Items', 'Nuts', 'Other Protein Sources',
       'Personal Care', 'Pet + Animal Care', 'Prepared Foods', 'Snacks',
       'The Garden Center']]=df[['SaleNom', 'Baked Goods', 'Beverages', 'Classes and Events',
       'Condiments + Sauces', 'Dairy', 'Dried Herbs + Spices', 'Eggs',
       'Grains, Flours, Cereal + Pastas', 'Handmade Home Goods + Gifts',
       'Honey, Syrups, Jams + Jellies', 'Iowa Food Co-op Shop',
       'Local Produce', 'Meat - Beef', 'Meat - Chicken + Capon', 'Meat - Pork',
       'Meats - Other', 'Non-Food Items', 'Nuts', 'Other Protein Sources',
       'Personal Care', 'Pet + Animal Care', 'Prepared Foods', 'Snacks',
       'The Garden Center']].fillna(0)

In [ ]:
cleaning_df = df[['IDCyc','IDMemb','Baked Goods', 'Beverages',
       'Classes and Events', 'Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center','SaleNom']]

In [ ]:
#Need to sort df oldest to newest first as we start making columns that cumulate over time.
cleaning_df = cleaning_df.sort_values(by=['IDMemb','IDCyc'], ascending=True)

In [ ]:
cleaning_df.head()

In [ ]:
cleaning_df[cleaning_df['IDMemb']==1078]

In [ ]:
#Creating a Cumulative Sum column to add customer orders as they go since we can't use total sales in our final model.
cleaning_df["Cumulative_Sum"] = cleaning_df.groupby("IDMemb")["SaleNom"].transform(lambda x: x.shift().fillna(0).cumsum())

In [ ]:
#Column of 1s and 0s for whether or not a person ordered during that ordering period
cleaning_df['Ordered'] = cleaning_df['SaleNom'].apply((lambda x: 1 if x>0 else 0))

In [ ]:
#Summing the new column to have a cumulative sum of total orders for each member.
cleaning_df['Cumulative_Number_of_Orders'] = cleaning_df.groupby("IDMemb")["Ordered"].transform(lambda x: x.shift().fillna(0).cumsum())

In [ ]:
#Calculating average order when the person orders. 
cleaning_df['order_per_cycle_when_ordering'] = cleaning_df['Cumulative_Sum']/cleaning_df['Cumulative_Number_of_Orders']

In [ ]:
column_list = ['Baked Goods', 'Beverages','Classes and Events','Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center']

for column in column_list:
    cleaning_df[f'{column}_Cum'] = cleaning_df.groupby("IDMemb")[column].transform(lambda x: x.shift().fillna(0).cumsum())
    cleaning_df.drop(column, axis=1, inplace=True)

In [ ]:
cleaning_df.head()

In [ ]:
cleaning_df[cleaning_df['IDMemb']==1016]

In [ ]:
cleaning_df.sort_values(by=['IDCyc','IDMemb'], ascending=True,inplace = True)

In [ ]:
#Cumulative count of how many cycles this person has been a member.
cleaning_df['Cycles_as_member'] = cleaning_df.groupby('IDMemb')['IDMemb'].cumcount() + 1

In [ ]:
cleaning_df.head()

In [ ]:
#Calulate how much each customer orders per cycle by dividing their total orders by the number of cycles they have been a member.
cleaning_df['order_per_cycle'] = cleaning_df['Cumulative_Sum']/(cleaning_df['Cycles_as_member']-1)

In [ ]:
cleaning_df.head()

In [ ]:
cleaning_df['order_per_cycle'] = cleaning_df['order_per_cycle'].fillna(0)

In [ ]:
cleaning_df['order_per_cycle_when_ordering'] = cleaning_df['order_per_cycle_when_ordering'].fillna(0)

In [ ]:
cleaning_df.head()

In [ ]:
cleaning_df.columns

In [ ]:
df.columns

In [ ]:
df.drop(['Baked Goods', 'Beverages',
       'Classes and Events', 'Condiments + Sauces', 'Dairy',
       'Dried Herbs + Spices', 'Eggs', 'Grains, Flours, Cereal + Pastas',
       'Handmade Home Goods + Gifts', 'Honey, Syrups, Jams + Jellies',
       'Iowa Food Co-op Shop', 'Local Produce', 'Meat - Beef',
       'Meat - Chicken + Capon', 'Meat - Pork', 'Meats - Other',
       'Non-Food Items', 'Nuts', 'Other Protein Sources', 'Personal Care',
       'Pet + Animal Care', 'Prepared Foods', 'Snacks', 'The Garden Center'],axis=1, inplace=True)

In [ ]:
print(df.shape,cleaning_df.shape)

In [ ]:
df.columns

In [ ]:
df=df.merge(cleaning_df,how='left', on =['IDCyc','IDMemb','SaleNom'])

In [ ]:
df.shape

In [ ]:
df = df.sort_values(['IDMemb','IDCyc'],ascending = True)

In [ ]:
filepath = '/Users/emilydanielbowser/Documents/Iowa Food Coop/Data/Intermediate Data/cleaned_data'
df.to_csv(filepath)